<a href="https://colab.research.google.com/github/ManelSoengas/NLP_Curs/blob/main/Utilitzant_Transformers__BPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Visió general de l'algorisme**

---


A continuació, ens endinsarem en els tres algorismes principals de tokenització de subparaules: BPE (utilitzat per GPT-2 i altres), WordPiece (utilitzat per exemple per BERT) i Unigram (utilitzat per T5 i altres). Abans de començar, aquí teniu una visió general ràpida de com funcionen cadascun. No dubteu a tornar a aquesta taula després de llegir cadascuna de les seccions següents si encara no té sentit per a vosaltres.

**Tokenització de codificació de parells de bytes**

---
BPE (Byte Pair Encoding) és un algorisme de compressió adaptat per tokenitzar textos. En lloc de separar el text en paraules senceres, divideix les paraules en fragments més petits (subwords) que poden recombinar-se per reconstruir moltes paraules diferents.

**Per què s’utilitza?**

1. Redueix la mida del vocabulari.

1. Permet manejar paraules desconegudes o noves.

1. És molt útil per a idiomes amb molta flexió (com el català, alemany, etc.).

1. Equilibra entre paraules senceres i caràcters individuals.


In [1]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

# Serveix per fer un conteig de freqüència de paraules pre-tokenitzades a partir d’un corpus de textos,
# utilitzant el pre-tokenitzador intern d’un fast tokenizer de Hugging Face

In [ ]:
# El següent pas és calcular el vocabulari base, format per tots els caràcters utilitzats en el corpus:

alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

In [5]:
vocab = ["<|endoftext|>"] + alphabet.copy()

# També afegim les fitxes especials utilitzades pel model al principi d'aquest vocabulari.
# En el cas de GPT-2, l'únic testimoni especial és "<|endoftext|>"

In [6]:
# Dividir cada paraula en caràcters individuals, per poder començar a entrenar

splits = {word: [c for c in word] for word in word_freqs.keys()}

In [7]:
# Funció que calcul1 la freqüència de cada parell.
# S'utilitza això a cada pas de la formació:

def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [ ]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

In [ ]:
# La parella més freqüent.

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

In [10]:
# primera fusió per aprendre és ('Ġ', 't') -> 'Ġt', i s'afegeix 'Ġt'al vocabulari:

merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [11]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [ ]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

In [13]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [ ]:
# Com a resultat, s'ha après 19 regles de combinació (el vocabulari inicial tenia una mida de 31 a 30 caràcters a l'alfabet, més el testimoni especial)
print(merges)

In [15]:
# Per tokenitzar un text nou, el pre-tokenitzem, el dividim i, a continuació, apliquem totes les regles de fusió apreses

def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [ ]:
tokenize("This is not a token.")